Note to self: I should defintely explore an object-oriented PyTorch approach in the future. It is annoying having to pass around every single required variable to helper functions

Import Statements

In [ ]:
import os
import torch
import torchvision
from torch import nn 
from torch.autograd import Variable
from torchvision import transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from torch.utils.data import Dataset
from mpl_toolkits.mplot3d import Axes3D
import scipy.io

Some Variables

In [ ]:
#number of intermediate steps
q = 500

#step size
h = 0.8

#start and end location
start_t = 0.1
end_t = start_t + h

N = None #number of x values for training

Import Data

In [ ]:
#CREDIT TO https://github.com/maziarraissi/PINNs/blob/master/appendix/discrete_time_inference%20(Burgers)/Burgers_systematic.py FOR THE BELOW CODE TO IMPORT IRK WEIGHTS AND THE LABEL DATA (AND FOR THE DATA ITSELF)

tmp = np.float32(np.loadtxt('./data/Butcher_IRK%d.txt' % (q), ndmin = 2))
IRK_weights = np.reshape(tmp[0:q**2+q], (q+1,q)) #presume the first q rows and q columns represent a_{ij} values, and the final row represents the b_j values
IRK_times = tmp[q**2+q:] #not sure what this is used for


data = scipy.io.loadmat('./data/burgers_shock.mat')

t_points = data['t'].flatten()[:,None] # T x 1
x_points = data['x'].flatten()[:,None] # N x 1
Exact = np.real(data['usol']).T # T x N

#I may choose to generate my own data once more...
'''#get data labels at t = 0.9 
index = np.where(t_points == end_t)[0][0]
end_labels = Exact[index, :]'''


#MAKE SURE WHEN CREATE INPUTS that REQUIRES_GRADE = True

Define Network

In [ ]:
class PINN(nn.Module):
    def __init__(self):
        super().__init__()
        self.act_func = nn.Tanh()
        self.predict = nn.Sequential(
            nn.Linear(1, 50),
            self.act_func,
            nn.Linear(50, 50),
            self.act_func,
            nn.Linear(50, 50),
            self.act_func,
            nn.Linear(50, q + 1)
        )

    def forward(self, x):
        x = self.predict(x)
        return x

Helper Methods

In [ ]:
#Nonelinear Burgers' Operator
def NLO(input, outputs): #outputs is of the form N x q. Inputs is of form N x 1 and was used to get outputs

    u = outputs
    u_x = torch.autograd.grad(u, input, grad_outputs=torch.ones_like(u), retain_graph=True, create_graph=True)[0]
    u_xx = torch.autograd.grad(u_x, input, grad_outputs=torch.ones_like(u_x), retain_graph=True, create_graph=True)[0]

    return u * u_x - (0.01/torch.pi)*u_xx
    
    


#Implicit Runge-Kutta Calculation (Combine Intermediate Terms with Butcher Tableau Coefficients)
def IRK(input, outputs, coefficients): #outputs is tensor of size Nx(q+1), represnting q stages for N x-values and the final n+1 prediction (which we ignore). coefficients is a tensor of size (q+1) x q representing the Butcher Tableau. We are not interested in its last row

    outputs = outputs[:-1] #ignore last row of outputs - that is the n+1 prediction

    global N
    global q
    global h

    coefficients_a = coefficients[:-1] #all but last row (all the a_{ij} coefficients). Shape (q,q)
    NLO_outputs = NLO(input, outputs) #perform non-linear operation on each of the outputs. Shape is still (N, q)
    
    N_coefficients_a = coefficients_a.repeat(N, 1, 1) #stack N coefficients_a tensors on top of each other
    NLO_outputs = NLO_outputs.unsqueeze(-1) #reshape NLO_outputs_reshaped into shape (N, q, 1) for matrix multiplication

    coefficients_applied = torch.bmm(N_coefficients_a, NLO_outputs) #perform matrix multiplication. This represents applying the coefficients a_{ij} from the Butcher Tableau to every element in NLO_outputs and THEN recombining them to make every new value which will go on to form the "calculated" outputs (intermediate stage values). Has shape (N, q, 1), since N_coefficients_a has shape (N, q, q) and NLO_outputs has shape (N, q, 1). For each layer in N, the matrix mulitplcaion (N, q) x (q, 1) takes place, resulting in a vector of shape (q, 1). N of these makes (N, q, 1)
    coefficients_applied = coefficients_applied.view(N, -1) #now shape (N, q)

    coefficients_applied = -h * coefficients_applied #apply the -h (-\delta t) to each element in coefficients_applied
    input = input.view(-1, 1) #reshape input from (N) to (N, 1)

    result = coefficients_applied + input #Every element in coefficients_applied has a value added to it which is the element from input on the same layer. We can add the shapes (N, q) and (N, 1) due to "broadcasting", which effectively stretches the shape of the vector

    return result #returns a result which is the same shape as "outputs" and represents the same intermediate values, except these are the "calculated" versions and not the "direct predictions"



Loss Functions

In [ ]:
def SSE_n(input, outputs, coefficients, network, device):

    coefficients = torch.tensor(coefficients).to(device)
    calculated_outputs = IRK(input, outputs, coefficients, network)

    SSE_n_loss = torch.sum((calculated_outputs - outputs)**2) + 

    return SSE_n_loss


def SSE_b():


def SEE():
    return SSE_n + SSE_b

Training Setup

Training

Evaluation

Graphing

Saving